In [17]:
import sys
!{sys.executable} -m pip install gpxpy
!{sys.executable} -m pip install osmnx
!{sys.executable} -m pip install networkx
!{sys.executable} -m pip install pyproj==3.2.1
!{sys.executable} -m pip install geopandas
!{sys.executable} -m pip install plotly

   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.4 MB 660.6 kB/s eta 0:00:25
   ---------------------------------------- 0.2/16.4 MB 2.4 MB/s eta 0:00:07
   - -------------------------------------- 0.6/16.4 MB 4.9 MB/s eta 0:00:04
   -- ------------------------------------- 1.2/16.4 MB 6.8 MB/s eta 0:00:03
   ---- ----------------------------------- 1.7/16.4 MB 7.8 MB/s eta 0:00:02
   ----- ---------------------------------- 2.1/16.4 MB 8.0 MB/s eta 0:00:02
   ------ --------------------------------- 2.6/16.4 MB 8.3 MB/s eta 0:00:02
   ------- -------------------------------- 3.2/16.4 MB 8.8 MB/s eta 0:00:02
   --------- ------------------------------ 3.7/16.4 MB 9.5 MB/s eta 0:00:02
   ---------- ----------------------------- 4.3/16.4 MB 9.4 MB/s eta 0:00:02
   ----------- ---------------------------- 4.8/16.4 MB 9.9 MB/s eta 0:00:02
   --------

In [ ]:
pip show pyproj osmnx networkx geopandas

In [7]:
import gpxpy

# Cargar el archivo GPX
with open('ruta.gpx', 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

# Extraer los puntos del track
points = []
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            points.append((point.latitude, point.longitude))

In [15]:
import gpxpy
import osmnx as ox
import networkx as nx
from collections import Counter
import folium
import math


# Load the GPX file
with open('ruta.gpx', 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

# Extract points from the GPX track
points = []
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            points.append((point.latitude, point.longitude))

# Function to get road type for a given point
def get_road_type(point, dist=500):  # Increase distance to 500 meters
    try:
        G = ox.graph_from_point(point, dist=dist, network_type='all')
        if len(G) == 0:
            raise nx.NetworkXPointlessConcept("Graph is empty.")
        for u, v, data in G.edges(data=True):
            if 'highway' in data:
                highway_type = data['highway']
                if isinstance(highway_type, list):
                    highway_type = highway_type[0]  # Take the first type if it's a list
                return highway_type
    except Exception as e:
        print(f"Error fetching OSM data for point {point}: {e}")
    return None

# Get road types for each point
road_types = []
for point in points:
    road_type = get_road_type(point,100)
    if road_type:  # Only add if road_type is not None
        road_types.append(road_type)

# Count the occurrences of each road type
road_type_counts = Counter(road_types)

# Calculate the percentage of each road type
total_points = len(road_types)
road_type_percentages = {road_type: (count / total_points) * 100 for road_type, count in road_type_counts.items()}

# Display the results
for road_type, percentage in road_type_percentages.items():
    print(f'{road_type}: {percentage:.2f}%')

# Define a color mapping based on road difficulty
road_color_map = {
    'motorway': 'blue',
    'trunk': 'blue',
    'primary': 'blue',
    'secondary': 'blue',
    'tertiary': 'green',
    'unclassified': 'gray',
    'residential': 'yellow',
    'service': 'lime',
    'track': 'darkorange',
    'path': 'red',
    'footway': 'red',
    'cycleway': 'red'
}

# Get road types for each point and store with coordinates
track_data = []
for point in points:
    road_type = get_road_type(point)
    if road_type:
        color = road_color_map.get(road_type, 'black')  # Default to black if type not found
        track_data.append((point[0], point[1], color))

# Initialize the map at the center of the track
map_center = [points[0][0], points[0][1]]
mymap = folium.Map(location=map_center, zoom_start=14)

# Add the track to the map
for i in range(len(track_data) - 1):
    folium.PolyLine(locations=[(track_data[i][0], track_data[i][1]), (track_data[i+1][0], track_data[i+1][1])], color=track_data[i][2], weight=5).add_to(mymap)

# Save the map to an HTML file
mymap.save('track_map.html')


Error fetching OSM data for point (39.839579, -0.371577): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.840378, -0.372854): Found no graph nodes within the requested polygon
Error fetching OSM data for point (39.840903, -0.375454): Found no graph nodes within the requested polygon
Error fetching OSM data for point (39.841439, -0.3765): Found no graph nodes within the requested polygon
Error fetching OSM data for point (39.84266, -0.381716): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.84576, -0.384149): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.84776, -0.384148): Found no graph nodes within the requested polygon
Error fetching OSM data for point (39.850793, -0.38707): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.852116, -0.39245): Found no graph nodes within the requested polygon
Error fetching OSM data for point (39.851318, -0.39

In [14]:
import gpxpy
import osmnx as ox
import networkx as nx
import folium
import math

# Load the GPX file
with open('ruta.gpx', 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

# Extract points and elevations from the GPX track
points = []
elevations = []
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            points.append((point.latitude, point.longitude))
            elevations.append(point.elevation)

def haversine(coord1, coord2):
    # Calculate the great-circle distance between two points
    # on the Earth's surface given their longitude and latitude.
    R = 6371000  # radius of Earth in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2.0) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    meters = R * c  # output distance in meters
    return meters

def calculate_bearing(coord1, coord2):
    # Calculate the bearing between two points
    lat1, lon1 = math.radians(coord1[0]), math.radians(coord1[1])
    lat2, lon2 = math.radians(coord2[0]), math.radians(coord2[1])
    delta_lon = lon2 - lon1
    x = math.sin(delta_lon) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(delta_lon)
    bearing = math.atan2(x, y)
    bearing = math.degrees(bearing)
    return (bearing + 360) % 360

# Detect curves and check if they are downhill
num_curves = 0
num_downhill_curves = 0
for i in range(1, len(points) - 1):
    bearing1 = calculate_bearing(points[i-1], points[i])
    bearing2 = calculate_bearing(points[i], points[i+1])
    change_in_bearing = abs(bearing2 - bearing1)
    if change_in_bearing >120:  # Threshold for detecting a curve, can be adjusted
        num_curves += 1
        if elevations[i] > elevations[i+1]:
            num_downhill_curves += 1

print(f'Número total de curvas: {num_curves}')
print(f'Número de curvas cuesta abajo: {num_downhill_curves}')

# Optionally, visualize the track with colored segments
def get_road_type(point, dist=500):  # Increase distance to 500 meters
    try:
        G = ox.graph_from_point(point, dist=dist, network_type='all')
        if len(G) == 0:
            raise nx.NetworkXPointlessConcept("Graph is empty.")
        for u, v, data in G.edges(data=True):
            if 'highway' in data:
                highway_type = data['highway']
                if isinstance(highway_type, list):
                    highway_type = highway_type[0]  # Take the first type if it's a list
                return highway_type
    except Exception as e:
        print(f"Error fetching OSM data for point {point}: {e}")
    return None

# Define a color mapping based on road difficulty
road_color_map = {
    'motorway': 'blue',
    'trunk': 'blue',
    'primary': 'blue',
    'secondary': 'blue',
    'tertiary': 'green',
    'unclassified': 'gray',
    'residential': 'yellow',
    'service': 'lime',
    'track': 'darkorange',
    'path': 'red',
    'footway': 'red',
    'cycleway': 'red'
}

# Get road types for each point and store with coordinates
track_data = []
for point in points:
    road_type = get_road_type(point)
    if road_type:
        color = road_color_map.get(road_type, 'black')  # Default to black if type not found
        track_data.append((point[0], point[1], color))

# Initialize the map at the center of the track
map_center = [points[0][0], points[0][1]]
mymap = folium.Map(location=map_center, zoom_start=14)

# Add the track to the map
for i in range(len(track_data) - 1):
    folium.PolyLine(locations=[(track_data[i][0], track_data[i][1]), (track_data[i+1][0], track_data[i+1][1])], color=track_data[i][2], weight=5).add_to(mymap)

# Save the map to an HTML file
mymap.save('track_map.html')

Número total de curvas: 148
Número de curvas cuesta abajo: 69
Error fetching OSM data for point (39.884379, -0.417974): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.923452, -0.378427): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.924514, -0.37306): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.858804, -0.342866): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.859989, -0.342037): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.86142, -0.343133): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.862036, -0.34473): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.862653, -0.343586): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.863355, -0.343267): Connectivity is undefined for the null graph.
Error

In [9]:
from collections import Counter

# Contar tipos de calzada
road_type_counts = Counter(road_types)

# Calcular porcentajes
total_points = len(road_types)
road_type_percentages = {road_type: count / total_points * 100 for road_type, count in road_type_counts.items()}

# Mostrar resultados
for road_type, percentage in road_type_percentages.items():
    print(f'{road_type}: {percentage:.2f}%')


secondary: 15.42%
track: 61.67%
unclassified: 2.50%
residential: 7.29%
path: 10.83%
service: 0.21%
living_street: 2.08%


In [ ]:
import gpxpy
import osmnx as ox
import networkx as nx
import folium
import math
from collections import Counter
import plotly.express as px

# Load the GPX file
with open('ruta.gpx', 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

# Extract points and elevations from the GPX track
points = []
elevations = []
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            points.append((point.latitude, point.longitude))
            elevations.append(point.elevation)

def haversine(coord1, coord2):
    # Calculate the great-circle distance between two points
    # on the Earth's surface given their longitude and latitude.
    R = 6371000  # radius of Earth in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2.0) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    meters = R * c  # output distance in meters
    return meters

def calculate_bearing(coord1, coord2):
    # Calculate the bearing between two points
    lat1, lon1 = math.radians(coord1[0]), math.radians(coord1[1])
    lat2, lon2 = math.radians(coord2[0]), math.radians(coord2[1])
    delta_lon = lon2 - lon1
    x = math.sin(delta_lon) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(delta_lon)
    bearing = math.atan2(x, y)
    bearing = math.degrees(bearing)
    return (bearing + 360) % 360

# Detect curves and check if they are downhill
num_curves = 0
num_downhill_curves = 0
for i in range(1, len(points) - 1):
    bearing1 = calculate_bearing(points[i-1], points[i])
    bearing2 = calculate_bearing(points[i], points[i+1])
    change_in_bearing = abs(bearing2 - bearing1)
    if change_in_bearing > 30:  # Threshold for detecting a curve, can be adjusted
        num_curves += 1
        if elevations[i] > elevations[i+1]:
            num_downhill_curves += 1

print(f'Número total de curvas: {num_curves}')
print(f'Número de curvas cuesta abajo: {num_downhill_curves}')

# Optionally, visualize the track with colored segments
def get_road_type(point, dist=500):  # Increase distance to 500 meters
    try:
        G = ox.graph_from_point(point, dist=dist, network_type='all')
        if len(G) == 0:
            raise nx.NetworkXPointlessConcept("Graph is empty.")
        for u, v, data in G.edges(data=True):
            if 'highway' in data:
                highway_type = data['highway']
                if isinstance(highway_type, list):
                    highway_type = highway_type[0]  # Take the first type if it's a list
                return highway_type
    except Exception as e:
        print(f"Error fetching OSM data for point {point}: {e}")
    return None

# Define a color mapping based on road difficulty
road_color_map = {
    'motorway': 'green',
    'trunk': 'lime',
    'primary': 'yellow',
    'secondary': 'orange',
    'tertiary': 'darkorange',
    'unclassified': 'red',
    'residential': 'red',
    'service': 'red',
    'track': 'red',
    'path': 'red',
    'footway': 'red',
    'cycleway': 'red'
}

# Get road types for each point and store with coordinates
track_data = []
for point in points:
    road_type = get_road_type(point)
    if road_type:
        color = road_color_map.get(road_type, 'black')  # Default to black if type not found
        track_data.append((point[0], point[1], color, road_type))

# Initialize the map at the center of the track
map_center = [points[0][0], points[0][1]]
mymap = folium.Map(location=map_center, zoom_start=14)

# Add the track to the map
for i in range(len(track_data) - 1):
    folium.PolyLine(locations=[(track_data[i][0], track_data[i][1]), (track_data[i+1][0], track_data[i+1][1])], color=track_data[i][2], weight=5).add_to(mymap)

# Save the map to an HTML file
mymap.save('track_map.html')

# Create a counter for the road types
road_type_counts = Counter([data[3] for data in track_data])

# Prepare data for the sunburst plot
labels = []
parents = []
values = []

for road_type, count in road_type_counts.items():
    labels.append(road_type)
    parents.append("")
    values.append(count)

# Create the sunburst plot
fig = px.sunburst(
    names=labels,
    parents=parents,
    values=values,
    title="Distribución de tipos de calzada"
)

# Show the plot
fig.show()


In [19]:
import gpxpy
import osmnx as ox
import networkx as nx
import folium
import math
from collections import Counter
import plotly.express as px
from concurrent.futures import ThreadPoolExecutor, as_completed

# Load the GPX file
with open('ruta.gpx', 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

# Extract points and elevations from the GPX track
points = []
elevations = []
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            points.append((point.latitude, point.longitude))
            elevations.append(point.elevation)

def haversine(coord1, coord2):
    R = 6371000  # radius of Earth in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2.0) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    meters = R * c  # output distance in meters
    return meters

def calculate_bearing(coord1, coord2):
    lat1, lon1 = math.radians(coord1[0]), math.radians(coord1[1])
    lat2, lon2 = math.radians(coord2[0]), math.radians(coord2[1])
    delta_lon = lon2 - lon1
    x = math.sin(delta_lon) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(delta_lon)
    bearing = math.atan2(x, y)
    bearing = math.degrees(bearing)
    return (bearing + 360) % 360

# Detect curves and check if they are downhill
num_curves = 0
num_downhill_curves = 0
for i in range(1, len(points) - 1):
    bearing1 = calculate_bearing(points[i-1], points[i])
    bearing2 = calculate_bearing(points[i], points[i+1])
    change_in_bearing = abs(bearing2 - bearing1)
    if change_in_bearing > 30:  # Threshold for detecting a curve, can be adjusted
        num_curves += 1
        if elevations[i] > elevations[i+1]:
            num_downhill_curves += 1

print(f'Número total de curvas: {num_curves}')
print(f'Número de curvas cuesta abajo: {num_downhill_curves}')

def get_road_type(point, dist=500):  # Increase distance to 500 meters
    try:
        G = ox.graph_from_point(point, dist=dist, network_type='all')
        if len(G) == 0:
            raise nx.NetworkXPointlessConcept("Graph is empty.")
        for u, v, data in G.edges(data=True):
            if 'highway' in data:
                highway_type = data['highway']
                if isinstance(highway_type, list):
                    highway_type = highway_type[0]  # Take the first type if it's a list
                return highway_type
    except Exception as e:
        print(f"Error fetching OSM data for point {point}: {e}")
    return None

# Define a color mapping based on road difficulty
road_color_map = {
    'motorway': 'gray',
    'trunk': 'gray',
    'primary': 'gray',
    'secondary': 'gray',
    'tertiary': 'gray',
    'unclassified': 'white',
    'residential': 'gray',
    'service': 'gray',
    'track': 'brown',
    'path': 'darkorange',
    'footway': 'red',
    'cycleway': 'red'
}

# Parallel processing to get road types for each point
def process_point(point):
    road_type = get_road_type(point)
    if road_type:
        color = road_color_map.get(road_type, 'black')
        return (point[0], point[1], color, road_type)
    return (point[0], point[1], 'black', None)

track_data = []
with ThreadPoolExecutor(max_workers=10) as executor:
    future_to_point = {executor.submit(process_point, point): point for point in points}
    for future in as_completed(future_to_point):
        track_data.append(future.result())

# Initialize the map at the center of the track
map_center = [points[0][0], points[0][1]]
mymap = folium.Map(location=map_center, zoom_start=14)

# Add the track to the map
for i in range(len(track_data) - 1):
    folium.PolyLine(locations=[(track_data[i][0], track_data[i][1]), (track_data[i+1][0], track_data[i+1][1])], color=track_data[i][2], weight=5).add_to(mymap)

# Save the map to an HTML file
mymap.save('track_map.html')

# Create a counter for the road types
road_type_counts = Counter([data[3] for data in track_data if data[3]])

# Prepare data for the sunburst plot
labels = []
parents = []
values = []

for road_type, count in road_type_counts.items():
    labels.append(road_type)
    parents.append("")
    values.append(count)

# Create the sunburst plot
fig = px.sunburst(
    names=labels,
    parents=parents,
    values=values,
    title="Distribución de tipos de calzada"
)

# Show the plot
fig.show()


Número total de curvas: 438
Número de curvas cuesta abajo: 206
Error fetching OSM data for point (39.884379, -0.417974): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.923452, -0.378427): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.924514, -0.37306): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.858804, -0.342866): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.859989, -0.342037): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.86142, -0.343133): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.862036, -0.34473): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.862653, -0.343586): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.863355, -0.343267): Connectivity is undefined for the null graph.
Erro

In [20]:
import gpxpy
import osmnx as ox
import networkx as nx
import folium
import math
from collections import Counter
import plotly.express as px
from concurrent.futures import ThreadPoolExecutor, as_completed

# Load the GPX file
with open('ruta.gpx', 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

# Extract points and elevations from the GPX track
points = []
elevations = []
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            points.append((point.latitude, point.longitude))
            elevations.append(point.elevation)

def haversine(coord1, coord2):
    R = 6371000  # radius of Earth in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2.0) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    meters = R * c  # output distance in meters
    return meters

def calculate_bearing(coord1, coord2):
    lat1, lon1 = math.radians(coord1[0]), math.radians(coord1[1])
    lat2, lon2 = math.radians(coord2[0]), math.radians(coord2[1])
    delta_lon = lon2 - lon1
    x = math.sin(delta_lon) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(delta_lon)
    bearing = math.atan2(x, y)
    bearing = math.degrees(bearing)
    return (bearing + 360) % 360

# Detect curves and check if they are downhill
num_curves = 0
num_downhill_curves = 0
for i in range(1, len(points) - 1):
    bearing1 = calculate_bearing(points[i-1], points[i])
    bearing2 = calculate_bearing(points[i], points[i+1])
    change_in_bearing = abs(bearing2 - bearing1)
    if change_in_bearing > 30:  # Threshold for detecting a curve, can be adjusted
        num_curves += 1
        if elevations[i] > elevations[i+1]:
            num_downhill_curves += 1

print(f'Número total de curvas: {num_curves}')
print(f'Número de curvas cuesta abajo: {num_downhill_curves}')

def get_road_type(point, dist=200):  # Increase distance to 500 meters
    try:
        G = ox.graph_from_point(point, dist=dist, network_type='all')
        if len(G) == 0:
            raise nx.NetworkXPointlessConcept("Graph is empty.")
        for u, v, data in G.edges(data=True):
            if 'highway' in data:
                highway_type = data['highway']
                if isinstance(highway_type, list):
                    highway_type = highway_type[0]  # Take the first type if it's a list
                return highway_type
    except Exception as e:
        print(f"Error fetching OSM data for point {point}: {e}")
    return None

# Define a color mapping based on road difficulty
road_color_map = {
    'motorway': 'gray',
    'trunk': 'gray',
    'primary': 'gray',
    'secondary': 'gray',
    'tertiary': 'gray',
    'unclassified': 'black',
    'residential': 'gray',
    'service': 'gray',
    'track': 'orange',
    'path': 'darkorange',
    'footway': 'red',
    'cycleway': 'red'
}

# Parallel processing to get road types for each point
def process_point(index, point):
    road_type = get_road_type(point)
    if road_type:
        color = road_color_map.get(road_type, 'black')
        return (index, point[0], point[1], color, road_type)
    return (index, point[0], point[1], 'black', None)

track_data = []
with ThreadPoolExecutor(max_workers=10) as executor:
    future_to_point = {executor.submit(process_point, idx, point): idx for idx, point in enumerate(points)}
    for future in as_completed(future_to_point):
        track_data.append(future.result())

# Sort track_data based on the original index to maintain order
track_data.sort()

# Initialize the map at the center of the track
map_center = [points[0][0], points[0][1]]
mymap = folium.Map(location=map_center, zoom_start=14)

# Add the track to the map
for i in range(len(track_data) - 1):
    folium.PolyLine(locations=[(track_data[i][1], track_data[i][2]), (track_data[i+1][1], track_data[i+1][2])], color=track_data[i][3], weight=5).add_to(mymap)

# Save the map to an HTML file
mymap.save('track_map.html')

# Create a counter for the road types
road_type_counts = Counter([data[4] for data in track_data if data[4]])

# Prepare data for the sunburst plot
labels = []
parents = []
values = []

for road_type, count in road_type_counts.items():
    labels.append(road_type)
    parents.append("")
    values.append(count)

# Create the sunburst plot
fig = px.sunburst(
    names=labels,
    parents=parents,
    values=values,
    title="Distribución de tipos de calzada"
)

# Show the plot
fig.show()


Número total de curvas: 438
Número de curvas cuesta abajo: 206
Error fetching OSM data for point (39.841439, -0.3765): Found no graph nodes within the requested polygonError fetching OSM data for point (39.84266, -0.381716): Connectivity is undefined for the null graph.

Error fetching OSM data for point (39.840378, -0.372854): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.84576, -0.384149): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.840903, -0.375454): Found no graph nodes within the requested polygon
Error fetching OSM data for point (39.852116, -0.39245): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.851318, -0.395229): Found no graph nodes within the requested polygon
Error fetching OSM data for point (39.852383, -0.396562): Found no graph nodes within the requested polygon
Error fetching OSM data for point (39.855608, -0.403122): Connectivity is undefined for the nu

In [21]:
import gpxpy
import osmnx as ox
import networkx as nx
import folium
import math
from collections import Counter
import plotly.express as px
from concurrent.futures import ThreadPoolExecutor, as_completed

# Load the GPX file
with open('ruta.gpx', 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

# Extract points and elevations from the GPX track
points = []
elevations = []
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            points.append((point.latitude, point.longitude))
            elevations.append(point.elevation)

def haversine(coord1, coord2):
    R = 6371000  # radius of Earth in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2.0) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    meters = R * c  # output distance in meters
    return meters

def calculate_bearing(coord1, coord2):
    lat1, lon1 = math.radians(coord1[0]), math.radians(coord1[1])
    lat2, lon2 = math.radians(coord2[0]), math.radians(coord2[1])
    delta_lon = lon2 - lon1
    x = math.sin(delta_lon) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(delta_lon)
    bearing = math.atan2(x, y)
    bearing = math.degrees(bearing)
    return (bearing + 360) % 360

# Detect curves and check if they are downhill
num_curves = 0
num_downhill_curves = 0
for i in range(1, len(points) - 1):
    bearing1 = calculate_bearing(points[i-1], points[i])
    bearing2 = calculate_bearing(points[i], points[i+1])
    change_in_bearing = abs(bearing2 - bearing1)
    if change_in_bearing > 30:  # Threshold for detecting a curve, can be adjusted
        num_curves += 1
        if elevations[i] > elevations[i+1]:
            num_downhill_curves += 1

print(f'Número total de curvas: {num_curves}')
print(f'Número de curvas cuesta abajo: {num_downhill_curves}')

def get_road_type(point, dist=500):  # Increase distance to 500 meters
    try:
        G = ox.graph_from_point(point, dist=dist, network_type='all')
        if len(G) == 0:
            raise nx.NetworkXPointlessConcept("Graph is empty.")
        nearest_node = ox.distance.nearest_nodes(G, point[1], point[0])
        for u, v, data in G.edges(nbunch=[nearest_node], data=True):
            if 'highway' in data:
                highway_type = data['highway']
                if isinstance(highway_type, list):
                    highway_type = highway_type[0]  # Take the first type if it's a list
                return highway_type
    except Exception as e:
        print(f"Error fetching OSM data for point {point}: {e}")
    return None

# Define a color mapping based on road difficulty
road_color_map = {
    'motorway': '#808080',  # gray
    'trunk': '#a9a9a9',     # darkgray
    'primary': '#c0c0c0',   # silver
    'secondary': '#d3d3d3', # lightgray
    'tertiary': '#dcdcdc',  # gainsboro
    'unclassified': '#f5f5f5', # whitesmoke
    'residential': '#f5f5f5', # whitesmoke
    'service': '#f5f5f5',   # whitesmoke
    'track': '#8b4513',     # saddlebrown
    'path': '#a0522d',      # sienna
    'footway': '#a0522d',   # sienna
    'cycleway': '#a0522d',  # sienna
    'pedestrian': 'red'     # red
}

# Parallel processing to get road types for each point
def process_point(index, point):
    road_type = get_road_type(point)
    if road_type:
        color = road_color_map.get(road_type, 'black')
        return (index, point[0], point[1], color, road_type)
    return (index, point[0], point[1], 'black', None)

track_data = []
with ThreadPoolExecutor(max_workers=10) as executor:
    future_to_point = {executor.submit(process_point, idx, point): idx for idx, point in enumerate(points)}
    for future in as_completed(future_to_point):
        track_data.append(future.result())

# Sort track_data based on the original index to maintain order
track_data.sort()

# Initialize the map at the center of the track
map_center = [points[0][0], points[0][1]]
mymap = folium.Map(location=map_center, zoom_start=14)

# Add the track to the map
for i in range(len(track_data) - 1):
    folium.PolyLine(locations=[(track_data[i][1], track_data[i][2]), (track_data[i+1][1], track_data[i+1][2])], color=track_data[i][3], weight=5).add_to(mymap)

# Save the map to an HTML file
mymap.save('track_map.html')

# Create a counter for the road types
road_type_counts = Counter([data[4] for data in track_data if data[4]])

# Prepare data for the sunburst plot
labels = []
parents = []
values = []

for road_type, count in road_type_counts.items():
    labels.append(road_type)
    parents.append("")
    values.append(count)

# Create the sunburst plot
fig = px.sunburst(
    names=labels,
    parents=parents,
    values=values,
    title="Distribución de tipos de calzada"
)

# Show the plot
fig.show()


Número total de curvas: 438
Número de curvas cuesta abajo: 206
Error fetching OSM data for point (39.884379, -0.417974): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.923452, -0.378427): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.924514, -0.37306): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.858804, -0.342866): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.859989, -0.342037): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.86142, -0.343133): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.862036, -0.34473): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.864237, -0.343482): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.862653, -0.343586): Connectivity is undefined for the null graph.
Erro

In [1]:
import gpxpy
import osmnx as ox
import networkx as nx
import folium
import math
from collections import Counter
import plotly.express as px
from concurrent.futures import ThreadPoolExecutor, as_completed

# Load the GPX file
with open('ruta.gpx', 'r') as gpx_file:
    gpx = gpxpy.parse(gpx_file)

# Extract points and elevations from the GPX track
points = []
elevations = []
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            points.append((point.latitude, point.longitude))
            elevations.append(point.elevation)

def haversine(coord1, coord2):
    R = 6371000  # radius of Earth in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2.0) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    meters = R * c  # output distance in meters
    return meters

def calculate_bearing(coord1, coord2):
    lat1, lon1 = math.radians(coord1[0]), math.radians(coord1[1])
    lat2, lon2 = math.radians(coord2[0]), math.radians(coord2[1])
    delta_lon = lon2 - lon1
    x = math.sin(delta_lon) * math.cos(lat2)
    y = math.cos(lat1) * math.sin(lat2) - math.sin(lat1) * math.cos(lat2) * math.cos(delta_lon)
    bearing = math.atan2(x, y)
    bearing = math.degrees(bearing)
    return (bearing + 360) % 360

def calculate_inclination(elev1, elev2, distance):
    return (elev2 - elev1) / distance if distance != 0 else 0

# Detect and classify curves, including those with high inclination
num_easy_curves = 0
num_normal_curves = 0
num_hard_curves = 0
num_downhill_hard_curves = 0

curves_inclination = {'easy': 0, 'normal': 0, 'hard': 0, 'downhill_hard': 0}
curves_angle = {'easy': 0, 'normal': 0, 'hard': 0}

inclinations = []

for i in range(1, len(points) - 1):
    bearing1 = calculate_bearing(points[i-1], points[i])
    bearing2 = calculate_bearing(points[i], points[i+1])
    change_in_bearing = abs(bearing2 - bearing1)
    if change_in_bearing > 180:
        change_in_bearing = 360 - change_in_bearing

    distance = haversine(points[i], points[i+1])
    inclination = calculate_inclination(elevations[i], elevations[i+1], distance)
    inclinations.append(inclination)

    if change_in_bearing < 60:
        num_easy_curves += 1
        curves_inclination['easy'] += 1
        curves_angle['easy'] += 1
    elif 60 <= change_in_bearing < 150:
        num_normal_curves += 1
        curves_inclination['normal'] += 1
        curves_angle['normal'] += 1
    else:
        num_hard_curves += 1
        curves_inclination['hard'] += 1
        curves_angle['hard'] += 1
        if inclination < -0.1:
            num_downhill_hard_curves += 1
            curves_inclination['downhill_hard'] += 1

print(f'Número de curvas fáciles: {num_easy_curves}')
print(f'Número de curvas normales: {num_normal_curves}')
print(f'Número de curvas difíciles: {num_hard_curves}')
print(f'Número de curvas difíciles cuesta abajo: {num_downhill_hard_curves}')

def get_road_type(point, dist=50):
    try:
        G = ox.graph_from_point(point, dist=dist, network_type='all')
        if len(G) == 0:
            raise nx.NetworkXPointlessConcept("Graph is empty.")
        nearest_node = ox.distance.nearest_nodes(G, point[1], point[0])
        for u, v, data in G.edges(nbunch=[nearest_node], data=True):
            if 'highway' in data:
                highway_type = data['highway']
                if isinstance(highway_type, list):
                    highway_type = highway_type[0]
                return highway_type
    except Exception as e:
        print(f"Error fetching OSM data for point {point}: {e}")
    return None

# Define a color mapping for road types
road_color_map = {
    'motorway': '#808080',  # gray
    'trunk': '#808080',     # gray
    'primary': '#808080',   # gray
    'secondary': '#808080', # gray
    'tertiary': '#808080',  # gray
    'unclassified': 'black',  
    'residential': '#808080', # whitesmoke
    'service': '#808080',   # whitesmoke
    'track': '#8b4513',     # saddlebrown
    'path': '#a0522d',      # sienna
    'footway': '#a0522d',   # sienna
    'cycleway': '#a0522d',  # sienna
    'pedestrian': 'red'     # red
}

# Parallel processing to get road types for each point
def process_point(index, point):
    road_type = get_road_type(point)
    if road_type:
        color = road_color_map.get(road_type, 'black')
        return (index, point[0], point[1], color, road_type)
    return (index, point[0], point[1], 'black', None)

track_data = []
with ThreadPoolExecutor(max_workers=10) as executor:
    future_to_point = {executor.submit(process_point, idx, point): idx for idx, point in enumerate(points)}
    for future in as_completed(future_to_point):
        track_data.append(future.result())

# Sort track_data based on the original index to maintain order
track_data.sort()

# Initialize the map at the center of the track
map_center = [points[0][0], points[0][1]]
mymap = folium.Map(location=map_center, zoom_start=14)

# Add the track to the map
for i in range(len(track_data) - 1):
    folium.PolyLine(locations=[(track_data[i][1], track_data[i][2]), (track_data[i+1][1], track_data[i+1][2])], color=track_data[i][3], weight=5).add_to(mymap)

# Save the map to an HTML file
mymap.save('track_map.html')

# Create a counter for the road types
road_type_counts = Counter([data[4] for data in track_data if data[4]])

# Prepare data for the sunburst plot (road types)
labels = []
parents = []
values = []

for road_type, count in road_type_counts.items():
    labels.append(road_type)
    parents.append("")
    values.append(count)

# Create the sunburst plot for road types
fig_road_types = px.sunburst(
    names=labels,
    parents=parents,
    values=values,
    title="Distribución de tipos de calzada"
)

# Show the plot
fig_road_types.show()

# Prepare data for the sunburst plot (curve inclination types)
curve_inclination_labels = []
curve_inclination_parents = []
curve_inclination_values = []

for curve_type, count in curves_inclination.items():
    curve_inclination_labels.append(curve_type)
    curve_inclination_parents.append("")
    curve_inclination_values.append(count)

# Create the sunburst plot for curve inclination types
fig_curve_inclination = px.sunburst(
    names=curve_inclination_labels,
    parents=curve_inclination_parents,
    values=curve_inclination_values,
    title="Distribución de tipos de curvas según inclinación"
)

# Show the plot
fig_curve_inclination.show()

# Prepare data for the sunburst plot (curve angle types)
curve_angle_labels = []
curve_angle_parents = []
curve_angle_values = []

for curve_type, count in curves_angle.items():
    curve_angle_labels.append(curve_type)
    curve_angle_parents.append("")
    curve_angle_values.append(count)

# Create the sunburst plot for curve angle types
fig_curve_angle = px.sunburst(
    names=curve_angle_labels,
    parents=curve_angle_parents,
    values=curve_angle_values,
    title="Distribución de tipos de curvas según ángulo"
)

# Show the plot
fig_curve_angle.show()

# Add markers for the difficult curves on a new map
curves_map = folium.Map(location=map_center, zoom_start=14)
for i in range(1, len(points) - 1):
    bearing1 = calculate_bearing(points[i-1], points[i])
    bearing2 = calculate_bearing(points[i], points[i+1])
    change_in_bearing = abs(bearing2 - bearing1)
    if change_in_bearing > 180:
        change_in_bearing = 360 - change_in_bearing

    distance = haversine(points[i], points[i+1])
    inclination = calculate_inclination(elevations[i], elevations[i+1], distance)

    if (change_in_bearing >= 150 and inclination < -0.1):
        folium.Marker(
            location=(points[i][0], points[i][1]),
            popup=f"Curva difícil\nÁngulo: {change_in_bearing:.2f}°\nInclinación: {inclination:.2f}",
            icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(curves_map)

# Save the curves map to an HTML file
curves_map.save('curves_map.html')


Número de curvas fáciles: 257
Número de curvas normales: 210
Número de curvas difíciles: 31
Número de curvas difíciles cuesta abajo: 8
Error fetching OSM data for point (39.884379, -0.417974): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.923452, -0.378427): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.924514, -0.37306): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.859989, -0.342037): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.862036, -0.34473): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.858804, -0.342866): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.86142, -0.343133): Connectivity is undefined for the null graph.
Error fetching OSM data for point (39.863355, -0.343267): Connectivity is undefined for the null graph.
Error fetching OSM data for point (3